In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
import timeit
from pyspark.sql import DataFrame
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql import Window
import os

In [2]:
os.environ["PYSPARK_SUBMIT_ARGS"] = "--jars /home/jovyan/jars/neo4j-connector-apache-spark_2.12-4.1.2_for_spark_3.jar pyspark-shell"

In [3]:
spark = SparkSession \
    .builder \
    .config("spark.jars", "/home/jovyan/jars/neo4j-connector-apache-spark_2.12-4.1.2_for_spark_3.jar") \
    .appName("My App") \
    .getOrCreate()

spark.version

22/07/18 21:17:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


'3.1.2'

In [4]:
print(spark.sparkContext._jsc.sc().listJars())

Vector(spark://974c720cafa6:40157/jars/neo4j-connector-apache-spark_2.12-4.1.2_for_spark_3.jar)


In [5]:
musicians_schema = T.StructType() \
    .add(T.StructField(nullable=False, dataType=T.IntegerType(), name="id")) \
    .add(T.StructField(nullable=True, dataType=T.StringType(), name="name"))
musicians_data = [(1, "Alex Lifeson"),
                  (2, "Neil Peart"),
                  (3, "Geddy Lee")]
musicians_df = spark \
    .createDataFrame(data=musicians_data, schema=musicians_schema)
musicians_df.show()

+---+------------+
| id|        name|
+---+------------+
|  1|Alex Lifeson|
|  2|  Neil Peart|
|  3|   Geddy Lee|
+---+------------+



In [6]:
musicians_df \
    .write \
    .format("org.neo4j.spark.DataSource") \
    .option("url", "bolt://neo4j:7687") \
    .option("authentication.type", "basic") \
    .option("authentication.basic.username", "neo4j") \
    .option("authentication.basic.password", "test") \
    .option("labels", ":Person:Musician") \
    .option("node.keys", "id") \
    .mode("Overwrite") \
    .save()